In [1]:
import sys
import nltk
import sklearn
import numpy as np
import pandas as pd

In [2]:
data_frame= pd.read_table("SMSSpamCollection",header= None, encoding ='utf-8') 
print(data_frame.head())

      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [3]:
#Preprocessing
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
classes= le.fit_transform(data_frame[0])

#replacing email, phonr No, money, webaddress, number, puntuation, more than single space, leading and trailing white space
emails= data_frame[1]

emails= emails.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$', 'emailaddress')
emails= emails.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')
emails= emails.str.replace(r'^£|\$', 'money')
emails= emails.str.replace(r'\d+(\.\d+)?','number')
emails= emails.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')
emails= emails.str.replace(r'[^\w\d\s]', ' ')
emails= emails.str.replace(r'\s+', ' ')
emails= emails.str.replace(r'^\s+|\s+?$', '')
emails= emails.str.lower()

# remove stop words from text messages
from nltk.corpus import stopwords
stop_words= list(stopwords.words('english'))
emails=emails.apply(lambda x: ' '.join(i for i in x.split() if i not in stop_words))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
emails=emails.apply(lambda x: ' '.join(lemmatizer.lemmatize(i) for i in x.split()))

print(emails)

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry number wkly comp win fa cup final t...
3                     u dun say early hor u c already say
4                     nah think go usf life around though
                              ...                        
5567    numbernd time tried number contact u u number ...
5568                          ü b going esplanade fr home
5569                                 pity mood suggestion
5570    guy bitching acted like interested buying some...
5571                                       rofl true name
Name: 1, Length: 5572, dtype: object


In [7]:
new=list()
from nltk.tokenize import sent_tokenize, word_tokenize
for i in emails:
    words= word_tokenize(i)
    for k in words: new.append(k)
freq_words = nltk.FreqDist( new)
freq_words.most_common() 
feature_words=list(freq_words.keys())[:1500]
def features(msg):
    words= word_tokenize(msg)
    features= {}
    for i in feature_words:
        features[i]= (i in words)
    return features
data_set= zip(emails, classes)
feature_msgs=[(features(emails), y) for(emails,y) in data_set]





In [22]:
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(feature_msgs, test_size = 0.30)
print(len(training))
print(len(testing))




3900
1672


In [23]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB
model = SklearnClassifier(MultinomialNB()).train(training)
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
test_features, labels = zip(*testing)
predictions= model.classify_many(test_features)
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1465
           1       0.90      0.95      0.92       207

    accuracy                           0.98      1672
   macro avg       0.95      0.97      0.96      1672
weighted avg       0.98      0.98      0.98      1672

